In [1]:
from cognite.client.data_classes import FileMetadataWrite
from cognite.client.data_classes.data_modeling import SpaceApply
from cognite.client.data_classes.data_modeling.cdm.v1 import CogniteFileApply
from cognite.client.exceptions import CogniteException
from IPython.display import display

from tests.auth_utils import get_toolkit_client

In [2]:
client = get_toolkit_client("dev-cluster.env", enable_set_pending_ids=True)

Found dev-cluster.env file in repository root. Loaded variables from dev-cluster.env file.


In [3]:
SPACE = "sp_upgrade"

In [4]:
client.data_modeling.spaces.apply(SpaceApply(space=SPACE))

,value
space,sp_upgrade
is_global,False
last_updated_time,2025-07-04 09:38:40.297000
created_time,2025-07-04 09:38:40.297000


# Delete Asset-Centric FileMetadata

In [9]:
file = FileMetadataWrite(
    external_id="ts_case4_delete_asset_centric",
    name="Case 4 Cascading Delete",
)

In [10]:
# Ensure the file does not exist before running the test
try:
    client.files.delete(external_id=file.external_id, ignore_unknown_ids=True)
except CogniteException:
    try:
        client.data_modeling.instances.delete(CogniteFileApply(space=SPACE, external_id=file.external_id).as_id())
    except CogniteException as e:
        print(f"\033[91mError\033[0m {e.code} {e.message!s}")
    else:
        print("\033[92mFile node deleted successfully\033[0m")
else:
    print(f"\033[92mFile with external_id {file.external_id} deleted successfully or did not exists\033[0m")

File with external_id ts_case4_delete_asset_centric deleted successfully or did not exists


In [11]:
created, _ = client.files.create(file)
created

,value
external_id,ts_case4_delete_asset_centric
name,Case 4 Cascading Delete
labels,[]
id,2935566945330951
created_time,2025-07-18 13:03:56.873000
last_updated_time,2025-07-18 13:03:56.873000
uploaded,False


In [12]:
filecontent = b"This is filecontent"

In [13]:
client.files.upload_content_bytes(filecontent, external_id=created.external_id)

,value
external_id,ts_case4_delete_asset_centric
name,Case 4 Cascading Delete
id,2935566945330951
created_time,2025-07-18 13:03:56.873000
last_updated_time,2025-07-18 13:03:56.873000
uploaded,False


In [14]:
try:
    updated = client.files.set_pending_ids((SPACE, file.external_id), external_id=file.external_id)
except CogniteException as e:
    print(f"\033[91mError\033[0m {e.code} {e.message!s}")
else:
    display(updated)

,value
external_id,ts_case4_delete_asset_centric
name,Case 4 Cascading Delete
id,2935566945330951
created_time,2025-07-18 13:03:56.873000
last_updated_time,2025-07-18 13:04:00.270000
uploaded,True
uploaded_time,2025-07-18 13:03:58.627000
pending_instance_id,"{'space': 'sp_upgrade', 'external_id': 'ts_cas..."


In [15]:
cognite_file = CogniteFileApply(
    space=SPACE,
    external_id=file.external_id,
    name=file.name,
)

In [16]:
created_dm = client.data_modeling.instances.apply(cognite_file).nodes
created_dm

,space,instance_type,external_id,version,was_modified,last_updated_time,created_time
0,sp_upgrade,node,ts_case4_delete_asset_centric,1,True,2025-07-18 13:04:02.817,2025-07-18 13:04:02.817


In [17]:
retrieved_ts = client.files.retrieve(external_id=file.external_id)
retrieved_ts

,value
external_id,ts_case4_delete_asset_centric
instance_id,"{'space': 'sp_upgrade', 'external_id': 'ts_cas..."
name,Case 4 Cascading Delete
id,2935566945330951
created_time,2025-07-18 13:03:56.873000
last_updated_time,2025-07-18 13:04:00.270000
uploaded,True
uploaded_time,2025-07-18 13:03:58.627000
pending_instance_id,"{'space': 'sp_upgrade', 'external_id': 'ts_cas..."


In [18]:
try:
    client.files.delete(external_id=file.external_id)
except CogniteException as e:
    print(f"\033[91mError\033[0m {e!s}")
else:
    print(f"Deleted Time Series with external_id: {file.external_id}")

Error Resource not found. This may also be due to insufficient access rights. | code: 403 | X-Request-ID: 13057b9d-3074-9aaa-93e4-cf7daf2f7479 | cluster: az-arn-dev-002
The API Failed to process some items.
Successful (2xx): []
Unknown (5xx): []
Failed (4xx): [ts_case4_delete_asset_centric, ...]
